# Práctica 2. Regresión para la predición de la calidad del vino

Preprocesamiento del dataset [**wine-quality**](http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/) y construcción de redes neuronales *Multilayer Perceptron* para predecir la calidad del vino.

**Autor**: Nicolás Cubero

**Fecha**: 6 de Abril de 2020

## Importar librerías

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score, r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop, Adam

## Carga de datos

Primeramente, procedemos a cargar los *datasets* desde los ficheros *csv*:

In [2]:
# Cargar los datos de los vinos blancos
white = pd.read_csv('winequality-white.csv', sep=';')

# Cargar los datos de los vinos tinto
red = pd.read_csv('winequality-red.csv', sep=';')

## Preprocesamiento de los datos

Una vez cargados los datos, se procede a preprocesar los datos y a prepararlos para alimentar a los modelos *MLP* de regresión en la fase de entrenamiento.

En primer lugar, unificamos ambos *datasets* (el *dataset* con instancias de vino blanco y el que contiene instancias de vino tinto).

In [3]:
# Definir el atributo "type" para identificar los tipos de vinos
white['type'] = 0
red['type'] = 1

# Unificamos ambos datasets en uno único
wines = red.append(white, ignore_index=True)

Se separa el conjunto de datos de la etiqueta:

In [10]:
# Tomar los datos
X = wines.iloc[:, :12]
X = X.drop('quality', axis=1)

# Tomar la etiqueta
y = wines['quality']

**Normalizamos** las variables para evitar que el orden de los valores de las variables provoque que una variable predomine más que otras:

In [11]:
# Utilidad para ejecutar la normalización
scaler = StandardScaler()

# Normalizar el conjunto de datos
X = scaler.fit_transform(X)

## Construcción de modelos

A continuación, usamos los datos preprocesados para entrenar y evaluar diferentes modelos basados en diferentes arquitecturas.

Cada modelo será entrenado y evaluado mediante *k-fold* estratificado usando un conjunto de particiones *K=5*.

Primeramente, definimos la arquitectura del modelo sobre la cual desarrollaremos los modelos.
Se trata de una arquitectura con una única capa oculta de 64 unidades:

In [18]:
def model_MLP():
    
    model = Sequential()
    
    # Primera capa oculta
    model.add(Dense(64, input_shape=(11,), activation='relu'))
    
    # Segunda capa oculta
    model.add(Dense(1))
    
    return model

Procedemos a realizar un entrenamiento con estos datos:

In [25]:
# Establecer la semilla generadora de números aleatorios
seed = 7
np.random.seed(seed)

# Realizar particionamiento k-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Para computar la media de los errores
mse_loss_list = []
mae_score_list = []
r2_score_list = []

for train, test in kfold.split(X, y):
    model = model_MLP() # Cargar arquitectura
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    
    # Ajustar el modelo
    model.fit(X[train], y[train], epochs=10, verbose=True, batch_size=1)
    
    # Evaluar el modelo
    mse_loss, mae_score = model.evaluate(X[test], y[test])
    
    mse_loss_list.append(mse_loss)
    mae_score_list.append(mae_score)
    
    # Evaluar la puntuación R2
    y_pred = model.predict(X[test])
    r2 = r2_score(y[test], y_pred)
    
    r2_score_list.append(r2)
    
    # Mostrar la puntuación
    print('Error MSE: ', mse_loss)
    print('Error MAE: ', mae_score)
    print('Puntuación R2: ', r2)

# Mostrar puntuación media
print('Error MSE medio: ', sum(mse_loss_list)/5)
print('Error MAE medio: ', sum(mae_score_list)/5)
print('Puntuación R2 media: ', sum(r2_score_list)/5)

Epoch 1/10
5197/5197 [==============================] - 24s 5ms/step - loss: 1.7338 - mae: 0.8795
Epoch 2/10
5197/5197 [==============================] - 24s 5ms/step - loss: 0.5902 - mae: 0.5866
Epoch 3/10
5197/5197 [==============================] - 24s 5ms/step - loss: 0.5430 - mae: 0.5697
Epoch 4/10
5197/5197 [==============================] - 24s 5ms/step - loss: 0.5293 - mae: 0.5620
Epoch 5/10
5197/5197 [==============================] - 24s 5ms/step - loss: 0.5157 - mae: 0.5532
Epoch 6/10
5197/5197 [==============================] - 24s 5ms/step - loss: 0.5135 - mae: 0.5533
Epoch 7/10
5197/5197 [==============================] - 24s 5ms/step - loss: 0.5125 - mae: 0.5563
Epoch 8/10
5197/5197 [==============================] - 24s 5ms/step - loss: 0.5033 - mae: 0.5485
Epoch 9/10
5197/5197 [==============================] - 24s 5ms/step - loss: 0.5018 - mae: 0.5479
Epoch 10/10
1300/1300 [==============================] - 0s 99us/step
Error MSE:  0.49032466164002053
Error MAE:  0.53

Los resultados obtenidos se recogen en la siguiente tabla:

| Partición | MSE     | MAE     | R2      |
| :-------: | :-----: | :-----: | :-----: |
|  K = 1    | 0.49032 | 0.53974 | 0.35777 |
|  K = 2    | 0.53844 | 0.57266 | 0.30077 |
|  K = 3    | 0.53466 | 0.55338 | 0.29686 |
|  K = 4    | 0.46497 | 0.53593 | 0.38931 |
|  K = 5    | 0.52833 | 0.56540 | 0.30806 |
|  media    | 0.51043 | 0.55334 | 0.33056 |

Procedemos a repetir el experimento considerando 30 épocas:

In [26]:
# Establecer la semilla generadora de números aleatorios
seed = 7
np.random.seed(seed)

# Realizar particionamiento k-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Para computar la media de los errores
mse_loss_list = []
mae_score_list = []
r2_score_list = []

for train, test in kfold.split(X, y):
    model = model_MLP() # Cargar arquitectura
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    
    # Ajustar el modelo
    model.fit(X[train], y[train], epochs=30, verbose=True, batch_size=1)
    
    # Evaluar el modelo
    mse_loss, mae_score = model.evaluate(X[test], y[test])
    
    mse_loss_list.append(mse_loss)
    mae_score_list.append(mae_score)
    
    # Evaluar la puntuación R2
    y_pred = model.predict(X[test])
    r2 = r2_score(y[test], y_pred)
    
    r2_score_list.append(r2)
    
    # Mostrar la puntuación
    print('Error MSE: ', mse_loss)
    print('Error MAE: ', mae_score)
    print('Puntuación R2: ', r2)

# Mostrar puntuación media
print('Error MSE medio: ', sum(mse_loss_list)/5)
print('Error MAE medio: ', sum(mae_score_list)/5)
print('Puntuación R2 media: ', sum(r2_score_list)/5)

Epoch 1/30
5197/5197 [==============================] - 6s 1ms/step - loss: 2.2065 - mae: 0.9546
Epoch 2/30
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5749 - mae: 0.5814
Epoch 3/30
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5326 - mae: 0.5631
Epoch 4/30
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5154 - mae: 0.5541
Epoch 5/30
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5032 - mae: 0.5473
Epoch 6/30
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5001 - mae: 0.5470
Epoch 7/30
5197/5197 [==============================] - 6s 1ms/step - loss: 0.4987 - mae: 0.5483
Epoch 8/30
5197/5197 [==============================] - 6s 1ms/step - loss: 0.4904 - mae: 0.5411
Epoch 9/30
5197/5197 [==============================] - 6s 1ms/step - loss: 0.4879 - mae: 0.5411
Epoch 10/30
5197/5197 [==============================] - 6s 1ms/step - loss: 0.4805 - mae: 0.5382
Epoch 11/30
5197/5197 [======

Considerando 30 épocas se obtuvieron los siguientes resultados


| Partición | MSE     | MAE     | R2      |
| :-------: | :-----: | :-----: | :-----: |
|  K = 1    | 0.49332 | 0.55073 | 0.35384 |
|  K = 2    | 0.53744 | 0.56698 | 0.29606 |
|  K = 3    | 0.53027 | 0.56850 | 0.30264 |
|  K = 4    | 0.58522 | 0.59624 | 0.23137 |
|  K = 5    | 0.52210 | 0.55641 | 0.31622 |
|  media    | 0.53367 | 0.56777 | 0.30027 |

Comparamos estos resultados con los obtenidos considerando 10 épocas:

| Épocas | MSE medio  | MAE medio | R2 medio |
| :----: | :--------: | :-------: | :------: |
|  10    | 0.51043 | 0.55334 | 0.33056 |
|  30    | 0.53367 | 0.56777 | 0.30027 |

Se observa que al incrementar el número de épocas de 10 a 30 el rendimiento del modelo empeora. La razón a este fenómeno se explicaría porque la ejecución de un mayor número de épocas permitiría al modelo iniciar y acentuar un sobreajuste a los datos de entrenamiento, con lo que disminuye su capacidad de generalización

### Incremento en el número de capas

A continuación, volvemos a repetir el primer conjunto de experimentos considerando una segunda capa oculta para la arquitectura del model

In [27]:
def model_deep_MLP():
    
    model = Sequential()
    
    # Primera capa oculta
    model.add(Dense(64, input_shape=(11,), activation='relu'))
    
    # Segunda capa oculta
    model.add(Dense(64, activation='relu'))
    
    # Tercera capa oculta
    model.add(Dense(1))
    
    return model

In [28]:
# Establecer la semilla generadora de números aleatorios
seed = 7
np.random.seed(seed)

# Realizar particionamiento k-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Para computar la media de los errores
mse_loss_list = []
mae_score_list = []
r2_score_list = []

for train, test in kfold.split(X, y):
    model = model_deep_MLP() # Cargar arquitectura
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    
    # Ajustar el modelo
    model.fit(X[train], y[train], epochs=10, verbose=True, batch_size=1)
    
    # Evaluar el modelo
    mse_loss, mae_score = model.evaluate(X[test], y[test])
    
    mse_loss_list.append(mse_loss)
    mae_score_list.append(mae_score)
    
    # Evaluar la puntuación R2
    y_pred = model.predict(X[test])
    r2 = r2_score(y[test], y_pred)
    
    r2_score_list.append(r2)
    
    # Mostrar la puntuación
    print('Error MSE: ', mse_loss)
    print('Error MAE: ', mae_score)
    print('Puntuación R2: ', r2)

# Mostrar puntuación media
print('Error MSE medio: ', sum(mse_loss_list)/5)
print('Error MAE medio: ', sum(mae_score_list)/5)
print('Puntuación R2 media: ', sum(r2_score_list)/5)

Epoch 1/10
5197/5197 [==============================] - 7s 1ms/step - loss: 1.3356 - mae: 0.8205
Epoch 2/10
5197/5197 [==============================] - 7s 1ms/step - loss: 0.6390 - mae: 0.6120
Epoch 3/10
5197/5197 [==============================] - 7s 1ms/step - loss: 0.5856 - mae: 0.5904
Epoch 4/10
5197/5197 [==============================] - 7s 1ms/step - loss: 0.5622 - mae: 0.5834
Epoch 5/10
5197/5197 [==============================] - 7s 1ms/step - loss: 0.5429 - mae: 0.5711
Epoch 6/10
5197/5197 [==============================] - 7s 1ms/step - loss: 0.5277 - mae: 0.5628
Epoch 7/10
5197/5197 [==============================] - 7s 1ms/step - loss: 0.5339 - mae: 0.5641
Epoch 8/10
5197/5197 [==============================] - 7s 1ms/step - loss: 0.5163 - mae: 0.5598
Epoch 9/10
5197/5197 [==============================] - 7s 1ms/step - loss: 0.5136 - mae: 0.5534
Epoch 10/10
1300/1300 [==============================] - 0s 26us/step
Error MSE:  0.4836191951770049
Error MAE:  0.537496507167

Considerando un mayor número de capas intermedias se obtienen los siguientes resultados:


| Partición | MSE     | MAE     | R2      |
| :-------: | :-----: | :-----: | :-----: |
|  K = 1    | 0.48362 | 0.53749 | 0.36656 |
|  K = 2    | 0.49986 | 0.54567 | 0.34528 |
|  K = 3    | 0.53106 | 0.55282 | 0.30161 |
|  K = 4    | 0.47095 | 0.53682 | 0.38144 |
|  K = 5    | 0.48487 | 0.53836 | 0.36522 |
|  media    | 0.49403 | 0.54221 | 0.35202 |

Comparamos estos resultados con los obtenidos considerando 10 épocas:

| Modelo    | MSE medio  | MAE medio | R2 medio |
| :-------: | :--------: | :-------: | :------: |
| Básico    | 0.51043    | 0.55334   | 0.33056  |
| Más capas | 0.49403    | 0.54221   | 0.35202  |

Se aprecia que, ante un mayor número de capas, el rendimiento del modelo mejora

### Incremento en el número de nodos

Se probará ahora, a desarrollar otro modelo basado en la primera arquitectura probando un mayor número de nodos en la capa intermedia

In [29]:
def model_larger_MLP():
    
    model = Sequential()
    
    # Primera capa oculta
    model.add(Dense(128, input_shape=(11,), activation='relu'))
    
    # Tercera capa oculta
    model.add(Dense(1))
    
    return model

In [30]:
# Establecer la semilla generadora de números aleatorios
seed = 7
np.random.seed(seed)

# Realizar particionamiento k-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Para computar la media de los errores
mse_loss_list = []
mae_score_list = []
r2_score_list = []

for train, test in kfold.split(X, y):
    model = model_larger_MLP() # Cargar arquitectura
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    
    # Ajustar el modelo
    model.fit(X[train], y[train], epochs=10, verbose=True, batch_size=1)
    
    # Evaluar el modelo
    mse_loss, mae_score = model.evaluate(X[test], y[test])
    
    mse_loss_list.append(mse_loss)
    mae_score_list.append(mae_score)
    
    # Evaluar la puntuación R2
    y_pred = model.predict(X[test])
    r2 = r2_score(y[test], y_pred)
    
    r2_score_list.append(r2)
    
    # Mostrar la puntuación
    print('Error MSE: ', mse_loss)
    print('Error MAE: ', mae_score)
    print('Puntuación R2: ', r2)

# Mostrar puntuación media
print('Error MSE medio: ', sum(mse_loss_list)/5)
print('Error MAE medio: ', sum(mae_score_list)/5)
print('Puntuación R2 media: ', sum(r2_score_list)/5)

Epoch 1/10
5197/5197 [==============================] - 6s 1ms/step - loss: 1.5018 - mae: 0.8192
Epoch 2/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5853 - mae: 0.5860
Epoch 3/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5503 - mae: 0.5727
Epoch 4/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5339 - mae: 0.5664
Epoch 5/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5176 - mae: 0.5543
Epoch 6/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5157 - mae: 0.5541
Epoch 7/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5153 - mae: 0.5569
Epoch 8/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5034 - mae: 0.5473
Epoch 9/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5008 - mae: 0.5476
Epoch 10/10
1300/1300 [==============================] - 0s 26us/step
Error MSE:  0.4839323749909034
Error MAE:  0.537194728851

Considerando un mayor número de nodos en la capa oculta se obtienen los siguientes resultados:


| Partición | MSE     | MAE     | R2      |
| :-------: | :-----: | :-----: | :-----: |
|  K = 1    | 0.48393 | 0.53719 | 0.36614 |
|  K = 2    | 0.52297 | 0.56122 | 0.31501 |
|  K = 3    | 0.52271 | 0.54975 | 0.31258 |
|  K = 4    | 0.46934 | 0.53538 | 0.38568 |
|  K = 5    | 0.53606 | 0.57357 | 0.29793 |
|  media    | 0.50700 | 0.55142 | 0.33505 |

Comparamos estos resultados con los obtenidos con el modelo original:

| Modelo       | MSE medio  | MAE medio | R2 medio |
| :----------: | :--------: | :-------: | :------: |
| Básico       | 0.51043    | 0.55334   | 0.33056  |
| Más unidades | 0.50700    | 0.55142   | 0.33505 |

En este caso, al incrementar el número de nodos, el rendimiento del nodo ha mejorado ligeramente.

Incrementar el número de nodos y/o el número de capas en la arquitectura resulta útil para permitir al modelo asimilar mayor cantidad de información del conjunto de datos de entrenamiento. No obstante, incrementar en exceso esta complejidad en la arquitectura puede tener 2 efectos negativos:

1. Incremento en el tiempo de cómputo necesario para entrenar el modelo y/o realizar prediciones con él.
2. Sobreajuste a los datos de entrenamiento (el modelo también aprendería el ruido contenido en el conjunto de datos), lo que perjudica a la capacidad de generalización del modelo.

### Reducción de la tasa de aprendizaje

Se prueba, a continuación, a desarrollar otra experimentación usando la arquitectura original, pero estableciendo una tasa de aprendizaje (*learning rate*) menor en el optimizador.

In [34]:
# Establecer la semilla generadora de números aleatorios
seed = 7
np.random.seed(seed)

# Cargar y configurar el optimizador
rmsprop = RMSprop(lr = 1e-4)

# Realizar particionamiento k-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Para computar la media de los errores
mse_loss_list = []
mae_score_list = []
r2_score_list = []

for train, test in kfold.split(X, y):
    model = model_MLP() # Cargar arquitectura
    model.compile(optimizer=rmsprop, loss='mse', metrics=['mae'])
    
    # Ajustar el modelo
    model.fit(X[train], y[train], epochs=10, verbose=True, batch_size=1)
    
    # Evaluar el modelo
    mse_loss, mae_score = model.evaluate(X[test], y[test])
    
    mse_loss_list.append(mse_loss)
    mae_score_list.append(mae_score)
    
    # Evaluar la puntuación R2
    y_pred = model.predict(X[test])
    r2 = r2_score(y[test], y_pred)
    
    r2_score_list.append(r2)
    
    # Mostrar la puntuación
    print('Error MSE: ', mse_loss)
    print('Error MAE: ', mae_score)
    print('Puntuación R2: ', r2)

# Mostrar puntuación media
print('Error MSE medio: ', sum(mse_loss_list)/5)
print('Error MAE medio: ', sum(mae_score_list)/5)
print('Puntuación R2 media: ', sum(r2_score_list)/5)

Epoch 1/10
5197/5197 [==============================] - 6s 1ms/step - loss: 10.8993 - mae: 2.6463
Epoch 2/10
5197/5197 [==============================] - 6s 1ms/step - loss: 1.4776 - mae: 0.9259
Epoch 3/10
5197/5197 [==============================] - 6s 1ms/step - loss: 1.1461 - mae: 0.8106
Epoch 4/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.9474 - mae: 0.7320
Epoch 5/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.8139 - mae: 0.6783
Epoch 6/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.7308 - mae: 0.6421
Epoch 7/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.6660 - mae: 0.6154
Epoch 8/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.6248 - mae: 0.5958
Epoch 9/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.5945 - mae: 0.5808
Epoch 10/10
1300/1300 [==============================] - 0s 25us/step
Error MSE:  0.5577689889761118
Error MAE:  0.57485109567

Con este descenso del *learning rate*, se han obtenido los siguientes resultados:


| Partición | MSE     | MAE     | R2      |
| :-------: | :-----: | :-----: | :-----: |
|  K = 1    | 0.55777 | 0.57485 | 0.26944 |
|  K = 2    | 0.66542 | 0.59577 | 0.12844 |
|  K = 3    | 0.60249 | 0.58305 | 0.20767 |
|  K = 4    | 0.57376 | 0.58635 | 0.24641 |
|  K = 5    | 0.58799 | 0.58877 | 0.22991 |
|  media    | 0.59749 | 0.58576 | 0.21637 |

Comparamos estos resultados con los obtenidos con el modelo original:

| Modelo       | MSE medio  | MAE medio | R2 medio |
| :----------: | :--------: | :-------: | :------: |
| Básico   | 0.51043    | 0.55334   | 0.33056  |
| Menor lr | 0.59749 | 0.58576 | 0.21637 |

Se aprecia que el rendimiento del modelo ha empeorado. Al disminuir la tasa de aprendizaje, el ajuste del modelo se desarrolla más lentamente, lo que provoca que el modelo necesite un mayor número de *epochs* para alcanzar el mismo óptimo local que el modelo original.

### Incremento de la tasa de aprendizaje

Por último, repetimos el experimento anterior incrementando el *learning rate*.

In [35]:
# Establecer la semilla generadora de números aleatorios
seed = 7
np.random.seed(seed)

# Cargar y configurar el optimizador
rmsprop = RMSprop(lr = 0.1)

# Realizar particionamiento k-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Para computar la media de los errores
mse_loss_list = []
mae_score_list = []
r2_score_list = []

for train, test in kfold.split(X, y):
    model = model_MLP() # Cargar arquitectura
    model.compile(optimizer=rmsprop, loss='mse', metrics=['mae'])
    
    # Ajustar el modelo
    model.fit(X[train], y[train], epochs=10, verbose=True, batch_size=1)
    
    # Evaluar el modelo
    mse_loss, mae_score = model.evaluate(X[test], y[test])
    
    mse_loss_list.append(mse_loss)
    mae_score_list.append(mae_score)
    
    # Evaluar la puntuación R2
    y_pred = model.predict(X[test])
    r2 = r2_score(y[test], y_pred)
    
    r2_score_list.append(r2)
    
    # Mostrar la puntuación
    print('Error MSE: ', mse_loss)
    print('Error MAE: ', mae_score)
    print('Puntuación R2: ', r2)

# Mostrar puntuación media
print('Error MSE medio: ', sum(mse_loss_list)/5)
print('Error MAE medio: ', sum(mae_score_list)/5)
print('Puntuación R2 media: ', sum(r2_score_list)/5)

Epoch 1/10
5197/5197 [==============================] - 6s 1ms/step - loss: 3.0003 - mae: 0.9733
Epoch 2/10
5197/5197 [==============================] - 6s 1ms/step - loss: 1.0805 - mae: 0.7464
Epoch 3/10
5197/5197 [==============================] - 6s 1ms/step - loss: 1.0766 - mae: 0.7398
Epoch 4/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.9181 - mae: 0.7164
Epoch 5/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.8609 - mae: 0.7096
Epoch 6/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.8264 - mae: 0.7094
Epoch 7/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.8253 - mae: 0.7100
Epoch 8/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.8079 - mae: 0.7063
Epoch 9/10
5197/5197 [==============================] - 6s 1ms/step - loss: 0.8977 - mae: 0.7082
Epoch 10/10
1300/1300 [==============================] - 0s 29us/step
Error MSE:  0.7717360491019029
Error MAE:  0.700252056121

Con este descenso del *learning rate*, se han obtenido los siguientes resultados:


| Partición | MSE     | MAE     | R2      |
| :-------: | :-----: | :-----: | :-----: |
|  K = 1    | 0.77174 | 0.70025 | -0.01081 |
|  K = 2    | 0.92076 | 0.68240 | -0.20601 |
|  K = 3    | 0.84066 | 0.76044 | -0.10556 |
|  K = 4    | 0.90805 | 0.71697 | -0.19265 |
|  K = 5    | 0.89405 | 0.74436 | -0.17091 |
|  media    | 0.86705 | 0.72088 | -0.13719 |

Comparamos estos resultados con los obtenidos con el modelo original:

| Modelo       | MSE medio  | MAE medio | R2 medio |
| :----------: | :--------: | :-------: | :------: |
| Básico   | 0.51043    | 0.55334   | 0.33056  |
| Mayor lr | 0.86705 | 0.72088 | -0.13719 |

Con bastante diferencia, se aprecia que el rendimiento del modelo ha empeorado respecto del modelo original y de los modelos anteriores.
Incrementar el *learning rate* en exceso povocaría oscilaciones en el ajuste del modelo que impedirían al mismo avanzar en la dirección del gradiente hacia los parámetros más óptimos.

De todos los modelos tratados, se concluye que el modelo con mayor número de capas intermedias presenta un rendimiento superior al resto.

# Clasificación del riesgo de abandono de los clientes de un banco

Ahora trataremos el uso de la redes neuronales *MLP* sobre un segundo problema: **La clasificación del riesgo de abandono de los clientes de un banco**.

Dado un *dataset* que recoge datos personales y bancarios de 10000 clientes, se pretende predecir si existe o no riesgo de abandono de estos clientes al banco al que pertenecen.

## Carga del *dataset*

Procedemos, en primer lugar, a cargar el *dataset*:

In [3]:
# Cargar el dataset
clients = pd.read_csv('Churn_Modelling.csv')

## Análisis exploratorio de datos

Analizamos brevemente el *dataset* cargado para obtener información preliminar sobre su estructura.

Conocer el tipo de dato de cada variable:

In [5]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


Calcular estadísticos básicos sobre cada variable numérica

In [6]:
clients.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Visualizar las 5 primeras filas del *dataset*

In [7]:
clients.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Preprocesamiento del *dataset*

A continuación, preprocesamos el *dataset* y lo particionaremos en los subconjuntos de entrenamiento y test.

Primeramente, separamos los datos de la etiqueta que se pretende predecir (*Exited*).Por otra parte, en los datos excluiremos las variables *row_number* y *CustomerId* al no resultar útiles para la predición.

In [11]:
X = clients.iloc[:,3:13].values
y = clients['Exited'].values

# Mostrar el conjunto de datos
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

Algunas de las variables del *dataset* constituyen variables categóricas que no pueden ser tratadas por las redes *MLP* de forma directa.

Se hace necesario **binarizar** estas variables categóricas (convertir cada variable categórica en tantas variables binarias como valores diferentes presente, establecer a 1 la variable binaria correspondiente al valor de la variable categórica original y establecer el resto de valores a 0). Ello se ejecutará con la utilidad *LabelEncoder*

In [12]:
# Binarizamos cada una de las variables categóricas
label_encoder_1 = LabelEncoder()
X[:, 1] = label_encoder_1.fit_transform(X[:, 1])

label_encoder_2 = LabelEncoder()
X[:, 2] = label_encoder_2.fit_transform(X[:, 2])

Dividimos el conjunto de entrenamiento en dos subconjuntos: *train* y *test*, reservando el 20% de las instancias para el conjunto de test.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Para terminar el preprocesamiento, **normalizamos** todas las variables del *dataset*:

In [20]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train) # Normalizar variables del conjunto de train
X_test = scaler.transform(X_test)        # Normalizar variables del conjunto de test

## Entrenamiento de modelos

Una vez preprocesados los datos, procedemos a definir la arquitectura del modelo y a ejecutar el entrenamiento.

In [21]:
def model_clients_MLP():
    
    model = Sequential()
    
    # Primera capa oculta
    model.add(Dense(6, activation='relu', input_shape=(10,)))
    
    # Segunda capa oculta
    model.add(Dense(6, activation='relu'))
    
    # Tercera capa oculta
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [22]:
# Preparar la arquitectura
model_clients = model_clients_MLP()
model_clients.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Ejecutar el entrenamiento:

In [41]:
model_clients.fit(X_train, y_train, epochs=100, batch_size=1, verbose=True)

Epoch 1/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.3327 - accuracy: 0.8604
Epoch 2/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.3319 - accuracy: 0.8648
Epoch 3/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.3330 - accuracy: 0.8621
Epoch 4/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.3316 - accuracy: 0.8609
Epoch 5/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.3333 - accuracy: 0.8606
Epoch 6/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.3329 - accuracy: 0.8616
Epoch 7/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.3332 - accuracy: 0.8610
Epoch 8/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.3333 - accuracy: 0.8610
Epoch 9/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.3321 - accuracy: 0.8625
Epoch 10/100
8000/8000 [==============================] - 13s 2m

8000/8000 [==============================] - 13s 2ms/step - loss: 0.3310 - accuracy: 0.8630
Epoch 80/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.3303 - accuracy: 0.8615
Epoch 81/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.3312 - accuracy: 0.8643
Epoch 82/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.3309 - accuracy: 0.8625
Epoch 83/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.3305 - accuracy: 0.8652
Epoch 84/100
8000/8000 [==============================] - 12s 2ms/step - loss: 0.3311 - accuracy: 0.8625
Epoch 85/100
8000/8000 [==============================] - 12s 1ms/step - loss: 0.3313 - accuracy: 0.8608
Epoch 86/100
8000/8000 [==============================] - 12s 2ms/step - loss: 0.3309 - accuracy: 0.8622
Epoch 87/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.3314 - accuracy: 0.8629
Epoch 88/100
8000/8000 [==============================] - 13s 2ms/st

Evaluamos el modelo:

In [42]:
y_pred = model_clients.predict(X_test, verbose=True)
y_pred = (y_pred>=0.5).astype('int64').squeeze()

# Evaluar la puntuación
score = model_clients.evaluate(X_test, y_test, verbose=True)

print('Accuracy del modelo: ', score[1])
print('Loss del modelo: ', score[0])

# Calcular la matriz de confusión
conf = confusion_matrix(y_test, y_pred)
print('Matriz de confusión:')
print(conf)

# Precisión del modelo
print('Precisión del modelo: {}'.format(precision_score(y_test, y_pred)))

# Recall del modelo
print('Recall del modelo: {}'.format(recall_score(y_test, y_pred)))

# F1-score del modelo
print('F1-score del modelo: {}'.format(f1_score(y_test, y_pred)))

# Cohen's kappa del modelo
print('Métrica Cohen\'s kappa del modelo: {}'.format(cohen_kappa_score(y_test, y_pred)))

2000/2000 [==============================] - 0s 25us/step
Accuracy del modelo:  0.19200000166893005
Loss del modelo:  51571.4255
Matriz de confusión:
[[   0 1616]
 [   0  384]]
Precisión del modelo: 0.192
Recall del modelo: 1.0
F1-score del modelo: 0.32214765100671144
Métrica Cohen's kappa del modelo: 0.0


Observamos que el rendimiento de este modelo es pésimo, se ha obtenido un clasificador trivial que clasifica todos los patrones como pertenencientes a la clase negativa. Analizando la evolución de las métricas de *accuracy* y *loss* medidas sobre el conjunto de entrenamiento, percibimos **ciertos efectos oscilatorios** durante el entrenamiento del modelo.

Repetimos el entrenamiento con la anterior arquitectura considerando un valor de *learning rate* de $10^{-5}$ para el optimizador Adam.

In [46]:
# Preparar la arquitectura
model_clients2 = model_clients_MLP()
model_clients2.compile(optimizer=Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_clients2.fit(X_train, y_train, epochs=100, batch_size=1, verbose=True)

Epoch 1/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.6666 - accuracy: 0.6410
Epoch 2/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.6282 - accuracy: 0.7400
Epoch 3/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.5943 - accuracy: 0.7729
Epoch 4/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.5654 - accuracy: 0.7856
Epoch 5/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.5414 - accuracy: 0.7890
Epoch 6/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.5223 - accuracy: 0.7900
Epoch 7/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.5077 - accuracy: 0.7901
Epoch 8/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.4969 - accuracy: 0.7920
Epoch 9/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.4890 - accuracy: 0.7925
Epoch 10/100
8000/8000 [==============================] - 13s 2m

8000/8000 [==============================] - 15s 2ms/step - loss: 0.4275 - accuracy: 0.8100
Epoch 80/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.4271 - accuracy: 0.8111
Epoch 81/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.4267 - accuracy: 0.8123
Epoch 82/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.4263 - accuracy: 0.8125
Epoch 83/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.4259 - accuracy: 0.8126
Epoch 84/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.4255 - accuracy: 0.8127
Epoch 85/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.4251 - accuracy: 0.8136
Epoch 86/100
8000/8000 [==============================] - 14s 2ms/step - loss: 0.4247 - accuracy: 0.8131
Epoch 87/100
8000/8000 [==============================] - 13s 2ms/step - loss: 0.4242 - accuracy: 0.8136
Epoch 88/100
8000/8000 [==============================] - 13s 2ms/st

In [47]:
y_pred = model_clients2.predict(X_test, verbose=True)
y_pred = (y_pred>=0.5).astype('int64').squeeze()

# Evaluar la puntuación
score = model_clients2.evaluate(X_test, y_test, verbose=True)

print('Accuracy del modelo: ', score[1])
print('Loss del modelo: ', score[0])

# Calcular la matriz de confusión
conf = confusion_matrix(y_test, y_pred)
print('Matriz de confusión:')
print(conf)

# Precisión del modelo
print('Precisión del modelo: {}'.format(precision_score(y_test, y_pred)))

# Recall del modelo
print('Recall del modelo: {}'.format(recall_score(y_test, y_pred)))

# F1-score del modelo
print('F1-score del modelo: {}'.format(f1_score(y_test, y_pred)))

# Cohen's kappa del modelo
print('Métrica Cohen\'s kappa del modelo: {}'.format(cohen_kappa_score(y_test, y_pred)))

2000/2000 [==============================] - 0s 26us/step
Accuracy del modelo:  0.737500011920929
Loss del modelo:  2604.3631162109373
Matriz de confusión:
[[1407  209]
 [ 316   68]]
Precisión del modelo: 0.24548736462093862
Recall del modelo: 0.17708333333333334
F1-score del modelo: 0.20574886535552195
Métrica Cohen's kappa del modelo: 0.053426416074081495


Observamos que el descenso de este *learning rate* ha permitido incrementar en gran medida el rendimiento del modelo.